# 第 8 章
# 对象引用、可变性和垃圾回收

---
本章先以一个比喻说明 Python 的变量:变量是标注,而不是盒子。如果你不知道引用式变量是什么,可以像这样对别人解释别名。  

然后,本章讨论对象标识、值和别名等概念。随后,本章会揭露元组的一个神奇特性:元组是不可变的,但是其中的值可以改变,之后就引申到浅复制和深复制。接下来的话题是引用和函数参数:可变的参数默认值导致的问题,以及如何安全地处理函数的调用者传入的可变参数。   

本章最后一节讨论垃圾回收、del 命令,以及如何使用弱引用“记住”对象,而无需对象本身存在。   

本章的内容有点儿枯燥,但是这些话题却是解决 Python 程序中很多不易察觉的 bug 的关键。  

首先,我们要抛弃变量是存储数据的盒子这一错误观念。  

---

### 8.1 变量不是盒子

Python 变量类似于 Java 中的引用式变量,因此最好把它们理解为附加在对象上的标注。

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
print(b)
# 如果把变量想象为盒子,那么无法解释 Python 中的赋值;应该把变量视作便利贴

[1, 2, 3, 4]


In [2]:
# 赋值语句的右边先执行
import traceback

class Gizmo:
    
    def __init__(self):
        print('Gizmo id: %d' % id(self))
        
x = Gizmo()    # 输出的 Gizmo id: ... 是创建 Gizmo 实例的副作用
try:
    y = Gizmo() * 10    # 这里表明,在尝试求积之前其实会创建一个新的 Gizmo 实例
except:                 
    traceback.print_exc()
    
dir()    # 但是,肯定不会创建变量 y,因为在对赋值语句的右边进行求值时抛出了异常

Gizmo id: 140659981755840
Gizmo id: 140659981755784


Traceback (most recent call last):
  File "<ipython-input-2-5e0ba6b11871>", line 11, in <module>
    y = Gizmo() * 10    # 这里表明,在尝试求积之前其实会创建一个新的 Gizmo 实例
TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'


['Gizmo',
 'In',
 'Out',
 '_',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'quit',
 'traceback',
 'x']

### 8.2 标识、相等性和别名

Lewis Carroll 是 Charles Lutwidge Dodgson 教授的笔名。Carroll 先生指的就是 Dodgson 教授,二者是同一个人。示例用 Python 表达了这个概念。

In [3]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles     # lewis 是 charles 的别名
print('lewis is charles: {}'.format(lewis is charles))
print('charles id({}), lewis id({})'.format(id(charles), id(lewis)))    # is 运算符和 id 函数确认了这一点
lewis['balance'] = 950    # 向 lewis 中添加一个元素相当于向 charles 中添加一个元素
print('charles: {}'.format(charles))

lewis is charles: True
charles id(140659981747016), lewis id(140659981747016)
charles: {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}


然而,假如有冒充者(姑且叫他 Alexander Pedachenko 博士)生于 1832年,声称他是 Charles L. Dodgson。这个冒充者的证件可能一样,但是 Pedachenko 博士不是 Dodgson 教授

In [4]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}# alex 指代的对象与赋值给 charles 的对象内容一样
print('alex == charles: {}'.format(alex == charles))# 比较两个对象,结果相等,这是因为 dict 类的 __eq__ 方法就是这样实现的
print('alex is not charles: {}'.format(alex is not charles))# 但它们是不同的对象。这是 Python 说明标识不同的方式:a is not b

alex == charles: True
alex is not charles: True


在上面段代码中,lewis 和 charles 是别名,即两个变量绑定同一个对象。而 alex 不是 charles 的别名,因为二者绑定的是不同的对象。alex 和 charles 绑定的对象具有相同的值(== 比较的就是值),但是它们的标识不同。

**在==和is之间选择**   

== 运算符比较两个对象的值(对象中保存的数据),而 is 比较对象的标识。

**元组的相对不可变性**

元组与多数 Python 集合(列表、字典、集,等等)一样,保存的是对象的引用。
如果引用的元素是可变的,即便元组本身不可变,元素依然可变。
也就是说,元组的不可变性其实是指 tuple 数据结构的物理内容(即保存的引用)不可变,与引用的对象无关。

In [5]:
t1 = (1, 2, [30, 40])    # t1 不可变,但是 t1[-1] 可变
t2 = (1, 2, [30, 40])    # 构建元组 t2,它的元素与 t1 一样
  
print('t1 == t2: {}'.format(t1 == t2))    # 虽然 t1 和 t2 是不同的对象,但是二者相等——与预期相符
print('t1[-1] id: {}'.format(id(t1[-1]))) # 查看 t1[-1] 列表的标识

t1[-1].append(99)                         # 就地修改 t1[-1] 列表
print('t1: {}'.format(t1))
print('t1[-1] id: {}'.format(id(t1[-1]))) # t1[-1] 的标识没变,只是值变了

print('t1 == t2: {}'.format(t1 == t2))    # 现在,t1 和 t2 不相等

t1 == t2: True
t1[-1] id: 140659981692232
t1: (1, 2, [30, 40, 99])
t1[-1] id: 140659981692232
t1 == t2: False


### 8.3 默认做浅复制

In [6]:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1)     # list(l1) 创建 l1 的副本

print('l2: {}'.format(l2))
print('l1 == l2: {}'.format(l1 == l2))    # 副本与源列表相等

# 但是二者指代不同的对象。对列表和其他可变序列来说,还能使用简洁的 l2 = l1[:] 语句创建副本
print('l2 is l1: {}'.format(l2 is l1))    

l2: [3, [55, 44], (7, 8, 9)]
l1 == l2: True
l2 is l1: False


In [7]:
# 为一个包含另一个列表的列表做浅复制

l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)    # l2 是 l1 的浅复制副本

l1.append(100)   # 把 100 追加到 l1 中,对 l2 没有影响
l1[1].remove(55) # 把内部列表 l1[1] 中的 55 删除。这对 l2 有影响,因为 l2[1] 绑定的列表与 l1[1] 是同一个

print('l1:', l1)
print('l2:', l2)

# 对可变的对象来说,如 l2[1] 引用的列表,+= 运算符就地修改列表。这次修改在 l1[1] 中也有体现,因为它是 l2[1] 的别名
l2[1] += [33, 22]

# 对元组来说,+= 运算符创建一个新元组,然后重新绑定给变量l2[2]。
# 这等同于 l2[2] = l2[2] + (10, 11)。现在,l1 和 l2 中最后位置上的元组不是同一个对象。
l2[2] += (10, 11) 

print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


**为任意对象做深复制和浅复制**

In [8]:
# 校车乘客在途中上车和下车

class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
            
    
    def pick(self, name):
        self.passengers.append(name)
        
        
    def drop(self, name):
        self.passengers.remove(name)
        
        
import copy

bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)       # 浅复制
bus3 = copy.deepcopy(bus1)   # 深复制

print('bus1 id: {}\nbus2 id: {}\nbus3 id: {}'.format(id(bus1), id(bus2), id(bus3)))

bus1.drop('Bill')            # bus1 中的 'Bill' 下车后,bus2 中也没有他了
print('bus2 passengers: {}'.format(bus2.passengers))

# 审查 passengers 属性后发现,bus1 和 bus2 共享同一个列表对象,因为 bus2 是 bus1 的浅复制副本
print('bus1 passengers id: {}\nbus2 passengers id: {}\nbus3 passengers id: {}'
      .format(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)))

print('bus3 passengers: {}'.format(bus3.passengers))  # bus3 是 bus1 的深复制副本,因此它的 passengers 属性指代另一个列表

bus1 id: 140659981801008
bus2 id: 140659981800952
bus3 id: 140659981800728
bus2 passengers: ['Alice', 'Claire', 'David']
bus1 passengers id: 140659981850760
bus2 passengers id: 140659981850760
bus3 passengers id: 140659981849096
bus3 passengers: ['Alice', 'Bill', 'Claire', 'David']


In [9]:
# 循环引用:b 引用 a,然后追加到 a 中;deepcopy 会想办法复制 a

a = [10, 20]
b = [a, 30]
a.append(b)
print('a: {}'.format(a))

c = copy.deepcopy(a)
print('c: {}'.format(c))

a: [10, 20, [[...], 30]]
c: [10, 20, [[...], 30]]


### 8.4 函数的参数作为引用时

Python 唯一支持的参数传递模式是共享传参(call by sharing)

共享传参指函数的各个形式参数获得实参中各个引用的副本。也就是说,函数内部的形参是实参的别名。

In [10]:
# 函数可能会修改接收到的任何可变对象

def f(a, b):
    a += b
    return a

x, y = 1, 2
print('f({}, {}) = {}'.format(x, y, f(x, y)))
print('x, y = {}, {}'.format(x, y))     # 数字 x 没变

a, b = [1, 2], [3, 4]
print('f([1, 2], [3, 4]) = {}'.format(f(a, b)))
print('a, b = {}, {}'.format(a, b))     # 列表 a 变了

t, u = (10, 20), (30, 40)
print('f({}, {}) = {}'.format(t, u, f(t, u)))
print('t, u = {}, {}'.format(t, u))     # 元组 t 没变

f(1, 2) = 3
x, y = 1, 2
f([1, 2], [3, 4]) = [1, 2, 3, 4]
a, b = [1, 2, 3, 4], [3, 4]
f((10, 20), (30, 40)) = (10, 20, 30, 40)
t, u = (10, 20), (30, 40)


**不要使用可变类型作为参数的默认值**  

可选参数可以有默认值,这是 Python 函数定义的一个很棒的特性,这样我们的 API 在进化的同时能保证向后兼容。  
然而,我们应该避免使用可变的对象作为参数的默认值。

In [11]:
# 一个简单的类,说明可变默认值的危险

class HauntedBus:
    """ 备受幽灵乘客折磨的校车 """
    
    def __init__(self, passengers=[]):    # 如果没传入 passengers 参数,使用默认绑定的列表对象,一开始是空列表
        self.passengers = passengers      # 这个赋值语句把 self.passengers 变成 passengers 的别名
                                          # 而没有传入 passengers 参数时,后者又是默认列表的别名
        
    def pick(self, name):
        self.passengers.append(name)      # 在 self.passengers 上调用 .remove() 和 .append() 方法时
                                          # 修改的其实是默认列表,它是函数对象的一个属性
        
        
    def drop(self, name):
        self.passengers.remove(name)
        
        
bus1 = HauntedBus(['Alice', 'Bill'])
print('bus1 passengers: {}'.format(bus1.passengers))
bus1.pick('Charlie')
bus1.drop('Alice')
print('bus1 passengers: {}'.format(bus1.passengers))    # 目前没什么问题,bus1 没有出现异常

bus2 = HauntedBus()                                     # 一开始,bus2 是空的,因此把默认的空列表赋值给 self.passengers
print('bus2 passengers: {}'.format(bus2.passengers))
bus2.pick('Carris')
print('bus2 passengers: {}'.format(bus2.passengers))

bus3 = HauntedBus()                                     # bus3 一开始也是空的,因此还是赋值默认的列表
print('bus3 passengers: {}'.format(bus3.passengers))    # 但是默认列表不为空
bus3.pick('Dave')
print('bus2 passengers: {}'.format(bus2.passengers))    # 登上 bus3 的 Dave 出现在 bus2 中

# 问题是,bus2.passengers 和 bus3.passengers 指代同一个列表
print('bus2 passengers is bus3 passengers: {}'.format(bus2.passengers is bus3.passengers))

# 但 bus1.passengers 是不同的列表
print('bus1 passengers: {}'.format(bus1.passengers))

# 问题在于,没有指定初始乘客的 HauntedBus 实例会共享同一个乘客列表

bus1 passengers: ['Alice', 'Bill']
bus1 passengers: ['Bill', 'Charlie']
bus2 passengers: []
bus2 passengers: ['Carris']
bus3 passengers: ['Carris']
bus2 passengers: ['Carris', 'Dave']
bus2 passengers is bus3 passengers: True
bus1 passengers: ['Bill', 'Charlie']


**防御可变参数**

如果定义的函数接收可变参数,应该谨慎考虑调用方是否期望修改传入的参数。

```python
# 一个简单的类,说明接受可变参数的风险

class TwilightBus:
    """ 让乘客销声匿迹的校车 """
    
    def __init__(self, passengers=None):
        if passengers is None:     
            self.passengers = []    # 这里谨慎处理,当 passengers 为 None 时,创建一个新的空列表
        else:
            self.passengers = passengers   # 然而,这个赋值语句把 self.passengers 变成 passengers 的别名
                                           # 而后者是传给 __init__ 方法的实参(即示例 8-14 中的basketball_team)的别名
            
    def pick(self, name):
        self.passengers.append(name)
        
    
    def drop(self, name):
        # 在 self.passengers 上调用 .remove() 和 .append() 方法其实会修改传给构造方法的那个列表
        self.passengers.remove(name)
```

这里的问题是,校车为传给构造方法的列表创建了别名。正确的做法是,校车自己维护乘客列表。  
修正的方法很简单:在 \__init__ 中,传入 passengers 参数时,应该把参数值的副本赋值给self.passengers

```python
def __init__(self, passengers=None):
    if passengers is None:
        self.passengers = []
    else:
        self.passengers = list(passengers)    # 创建 passengers 列表的副本;如果不是列表,就把它转换成列表
```

### 8.5 del和垃圾回收

del 语句删除名称,而不是对象。   
del 命令可能会导致对象被当作垃圾回收,但是仅当删除的变量保存的是对象的最后一个引用,或者无法得到对象时。   
重新绑定也可能会导致对象的引用数量归零,导致对象被销毁。   

In [12]:
# 为了演示对象生命结束时的情形,使用 weakref.finalize 注册一个回调函数,在销毁对象时调用

import weakref

s1 = {1, 2, 3}
s2 = s1     # s1 和 s2 是别名,指向同一个集合,{1, 2, 3}


def bye():  # 这个函数一定不能是要销毁的对象的绑定方法,否则会有一个指向对象的引用
    print('Gone with the wind...')
    
    
ender = weakref.finalize(s1, bye)  # 在 s1 引用的对象上注册 bye 回调

print('ender.alive:', ender.alive) # 调用 finalize 对象之前,.alive 属性的值为 True

del s1                             # 如前所述,del 不删除对象,而是删除对象的引用
print('ender.alive:', ender.alive)

# 重新绑定最后一个引用 s2,让 {1, 2, 3} 无法获取。对象被销毁了,调用了 bye 回调,ender.alive 的值变成了 False
s2 = 'spam'
print('ender.alive:', ender.alive)

ender.alive: True
ender.alive: True
Gone with the wind...
ender.alive: False


示例的目的是明确指出 del 不会删除对象,但是执行 del 操作后可能会导致对象不可获取,从而被删除。  
为什么示例中的 {1, 2, 3} 对象被销毁了?毕竟,我们把 s1 引用传给 finalize 函数了,而为了监控对象和调用回调,必须要有引用。  
这是因为,finalize 持有 {1, 2, 3} 的弱引用

### 8.6 弱引用

正是因为有引用,对象才会在内存中存在。当对象的引用数量归零后,垃圾回收程序会把对象销毁。但是,有时需要引用对象,而不让对象存在的时间超过所需时间。这经常用在缓存中。  

弱引用不会增加对象的引用数量。引用的目标对象称为所指对象 (referent)。因此我们说,弱引用不会妨碍所指对象被当作垃圾回收。  

弱引用在缓存应用中很有用,因为我们不想仅因为被缓存引用着而始终保存缓存对象。

In [13]:
# 弱引用是可调用的对象,返回的是被引用的对象;如果所指对象不存在了,返回 None

import weakref

a_set = {0, 1}
wref = weakref.ref(a_set)
print('wref()=',wref())
print('wref:', wref)

a_set = {2, 3, 4}
print('wref()=',wref())

print('wref() is None:', wref() is None)
print('wref() is None:', wref() is None)

wref()= {0, 1}
wref: <weakref at 0x7fedf4402a98; to 'set' at 0x7fedfbf5c908>
wref()= None
wref() is None: True
wref() is None: True
